In [37]:
from typing import List, Dict


def calculate_production_plan(load: float, fuels: Dict[str, float], powerplants: List[Dict[str, any]]) -> Dict[str, float]:
    # Sort powerplants based on cost and efficiency
    sorted_powerplants = sorted(powerplants, key=lambda p: (p['type'] != 'windturbine', p['efficiency'], p['pmin'], -p['pmax']))
    print(json.dumps(sorted_powerplants, sort_keys=True, indent=4))

    # Initialize variables
    remaining_load = load
    production_plan = {}

    # Iterate over powerplants and allocate power based on merit-order
    for powerplant in sorted_powerplants:
        pmax = powerplant['pmax']
        pmin = powerplant['pmin']
        print(powerplant["type"])

        # Calculate the maximum power that can be allocated for the powerplant
        max_power = min(remaining_load, pmax)

        # Calculate the power to be allocated while considering pmin
        allocated_power = max(max_power, pmin)

        # Update the remaining load and power production plan
        remaining_load -= allocated_power
        production_plan[powerplant['name']] = allocated_power

        # Break the loop if the remaining load is zero
        if remaining_load <= 0:
            break

    # If there is still a remaining load, allocate it to wind turbines
    if remaining_load > 0:
        wind_percentage = fuels.get('wind', 0) / 100
        wind_power = remaining_load / wind_percentage if wind_percentage > 0 else 0
        production_plan['wind'] = wind_power

    return production_plan


# Maintenant il faut fait un backward à cause  de l'exemple payload 6

In [38]:
import json
with open('../tests/payloads/payload11.json', 'r') as f:
    data = json.load(f)
    #print(get_checks(data))
    load = data['load']
    fuels = data['fuels']
    powerplants = data['powerplants']
    print(json.dumps(calculate_production_plan(load, fuels, powerplants), sort_keys=True, indent=4))


[
    {
        "efficiency": 1,
        "name": "windpark2",
        "pmax": 150,
        "pmin": 0,
        "type": "windturbine"
    },
    {
        "efficiency": 1,
        "name": "tj1",
        "pmax": 460,
        "pmin": 100,
        "type": "turbojet"
    },
    {
        "efficiency": 1,
        "name": "gasfiredbig2",
        "pmax": 460,
        "pmin": 100,
        "type": "gasfired"
    }
]
windturbine
turbojet
{
    "tj1": 150,
    "windpark2": 150
}
